In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import to_date, to_timestamp
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as F

In [ ]:
# inicio sessão spark
spark = pyspark.sql.SparkSession.builder.appName("consultas")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .getOrCreate()

In [ ]:
# lendo dados dos usuarios na bronze em formato parquet
df_bronze = spark.read.format("parquet")\
            .option("inferSchema", "true")\
            .option("header", "true")\
            .option("versionAsOf", "0")\
            .load("./data-lake/bronze/*.parquet")

In [ ]:
df_bronze.count()

In [ ]:
# lendo dados dos usuarios na silver em formato delta
df_silver = spark.read.format("delta")\
            .option("inferSchema", "true")\
            .option("header", "true")\
            .option("versionAsOf", "0")\
            .load("./data-lake/silver/")

In [ ]:
df_silver.count()

In [ ]:
# encerra sessao spark
spark.stop()